In [ ]:
import io
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

API_KEY = os.getenv("RENEWABLE_NINJA_API")


In [ ]:
API_BASE = 'https://www.renewables.ninja/api/'

s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + API_KEY}

URL = API_BASE + 'data/wind'

In [ ]:
def get_data(url, lat, lon, start_date, end_date):
    
	args = {
    'lat': lat,           
    'lon': lon,           
    'date_from': start_date,
    'date_to': end_date,
	
	# std options

    'capacity': 1.0,
    'height': 100,           
    'turbine': 'Vestas V80 2000', 
    'format': 'json',
    'mean' : 'month',
    'raw': 'true'
	}
	r = s.get(url, params=args)
	print(r.status_code)

	json_string = json.dumps(r.json()['data'])
	json_data = io.StringIO(json_string)

	data = pd.read_json(json_data, 
					 orient='index').drop('electricity', axis=1)

	return data


In [ ]:
data['estado'] = 'estado'
data.head()

In [ ]:
coordinates = pd.read_excel("../data/raw/state_coordinates.xlsx")


In [ ]:
coordinates.head()

In [ ]:
start_date = '2024-01-01'
end_date = '2024-12-31'

In [ ]:
wind_speed = pd.DataFrame()

In [ ]:
for i in range(0, len(coordinates)):
    print(i)
    estado = coordinates['Estado'][i]
    lat = coordinates['Latitude'][i]
    lon = coordinates['Longitude'][i]
    data = get_data(URL, lat, lon, start_date, end_date)
    data['estado'] = estado
    data['lat'] = lat
    data['lon'] = lon
    wind_speed = pd.concat([wind_speed, data])

In [ ]:
wind_speed = wind_speed.reset_index().rename(columns={'index':'data'})

In [ ]:
wind_speed['data'] = pd.to_datetime(wind_speed['data'])

In [ ]:
teste = wind_speed.groupby(['data', 'estado'])['wind_speed'].mean()

In [ ]:
teste = teste.reset_index().sort_values(by = 'estado').reset_index()

In [ ]:
teste.drop(['index'],axis=1, inplace=True)

In [ ]:
teste[teste['estado'] == 'Alagoas']

In [ ]:
wind_speed = pd.read_excel('../data/raw/wind_speed.xlsx')
wind_speed.head()

In [ ]:
columns = wind_speed.drop('Data', axis=1).columns.to_list()

In [ ]:
columns

In [ ]:
wind_speed2 = pd.melt(wind_speed, id_vars=['Data'], value_vars=columns, 
                      var_name = 'estado', value_name='y')
wind_speed2

In [ ]:
wind_2024 = teste.copy()

In [ ]:
wind_2024 = wind_2024.rename(columns = {
    'data': 'Data',
    'wind_speed':'y'
})

In [ ]:
wind_speed2 = pd.concat([wind_speed2, wind_2024])

In [ ]:
wind_2024 = pd.read_excel('../data/raw/wind_speed_2024.xlsx')

In [ ]:
wind_speed2['Data'] = pd.to_datetime(wind_speed2['Data'])

In [ ]:
wind_2024.info()

In [ ]:
wind_2024.sort_values(by = 'estado', inplace = True)

In [ ]:
wind_2024.sort_index()

In [ ]:
wind_2024 = wind_2024[['Data', 'estado', 'y']]
wind_2024.sort_values(by = 'Data')

In [ ]:
wind_2024.sort_values(by=['Data'])

In [ ]:
df_pivot = wind_2024.pivot(index='Data', columns='estado', values='y')

In [ ]:
df_pivot = df_pivot.reset_index()

In [ ]:
df_pivot.columns.name = None

In [ ]:
df_pivot

In [ ]:
df_pivot.to_excel('../data/raw/wind_speed_2024.xlsx')